-----------------------------------------------------------------------------

<h1> MOFs CO2 working capacity prediction with XGBoost </h1>

------------------------------------------------------------

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.display import display
from sklearn.model_selection import train_test_split
from
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import mean_absolute_error

<h2>loading dataset</h2>

In [6]:
df = pd.read_csv('Datasets/cif_xyz_nonfg_train.csv');df

,MOFname,volume,density,weight,surface_area,void_fraction,void_volume,functional_groups,metal_linker,organic_linker1,...,Lattice7,Lattice8,Lattice9,C+O+H,C,O,H,sumatom,mean_charge,sum_charge
0,mof_unit_2,2769.503842,1.326090,2211.697211,603.6100,0.13794,0.1040,F-OMe,10,44,...,-0.600769,-3.410680,18.641128,173,71,35,67,194,5.150000e-09,0.000001
1,mof_unit_3,1089.818728,1.178856,773.687960,788.5000,0.14874,0.1262,OMe-COOH,2,22,...,0.010529,0.148108,10.630959,80,32,14,34,82,-3.660000e-08,-0.000003
2,mof_unit_4,2205.198301,0.982408,1304.638720,1441.5300,0.21814,0.2220,H-SO3H,9,17,...,-0.078915,-6.446998,18.164659,107,56,23,28,112,4.460000e-08,0.000005
3,mof_unit_6,3954.659761,0.647909,1543.027680,2430.5500,0.37094,0.5725,Pr-NO2,9,7,...,-2.470500,-0.012234,22.470616,158,72,24,62,164,-1.830000e-08,-0.000003
4,mof_unit_7,3565.914939,0.910268,1954.749656,1530.0200,0.33337,0.3662,NH2,10,53,...,0.012840,-4.025383,22.428988,165,68,28,69,182,2.200000e-08,0.000004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68579,mof_unit_68607,949.067112,1.318868,753.787520,164.5781,0.00000,0.0000,Pr,2,10,...,4.211774,-0.032201,15.139554,84,38,8,38,86,-3.490000e-08,-0.000003
68580,mof_unit_68608,1202.182553,1.440028,1042.538240,161.0412,0.00000,0.0000,Ph-Cl,3,12,...,-0.709510,-0.700082,11.090143,100,56,8,36,104,3.850000e-08,0.000004
68581,mof_unit_68609,1188.302573,1.399781,1001.700216,150.9874,0.00000,0.0000,Pr-F,3,4,...,-0.398622,-0.155104,10.184892,110,47,8,55,119,3.360000e-08,0.000004
68582,mof_unit_68610,1506.660363,1.645811,1493.296496,161.4001,0.01108,0.0000,SO3H,10,42,...,-2.389941,-1.681765,14.747027,115,38,27,50,126,-5.560000e-08,-0.000007


seperate label column

In [7]:
# df = df[df['void_fraction'] >= 0]
# df.reset_index(inplace=True)
# df

In [8]:
df_label = df['CO2_working_capacity']
df = df.drop(['CO2_working_capacity','MOFname'],axis=1)
print(df.shape,' : ',df_label.shape);df

(68584, 35)  :  (68584,)


,volume,density,weight,surface_area,void_fraction,void_volume,functional_groups,metal_linker,organic_linker1,organic_linker2,...,Lattice7,Lattice8,Lattice9,C+O+H,C,O,H,sumatom,mean_charge,sum_charge
0,2769.503842,1.326090,2211.697211,603.6100,0.13794,0.1040,F-OMe,10,44,57,...,-0.600769,-3.410680,18.641128,173,71,35,67,194,5.150000e-09,0.000001
1,1089.818728,1.178856,773.687960,788.5000,0.14874,0.1262,OMe-COOH,2,22,24,...,0.010529,0.148108,10.630959,80,32,14,34,82,-3.660000e-08,-0.000003
2,2205.198301,0.982408,1304.638720,1441.5300,0.21814,0.2220,H-SO3H,9,17,24,...,-0.078915,-6.446998,18.164659,107,56,23,28,112,4.460000e-08,0.000005
3,3954.659761,0.647909,1543.027680,2430.5500,0.37094,0.5725,Pr-NO2,9,7,23,...,-2.470500,-0.012234,22.470616,158,72,24,62,164,-1.830000e-08,-0.000003
4,3565.914939,0.910268,1954.749656,1530.0200,0.33337,0.3662,NH2,10,53,55,...,0.012840,-4.025383,22.428988,165,68,28,69,182,2.200000e-08,0.000004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68579,949.067112,1.318868,753.787520,164.5781,0.00000,0.0000,Pr,2,10,18,...,4.211774,-0.032201,15.139554,84,38,8,38,86,-3.490000e-08,-0.000003
68580,1202.182553,1.440028,1042.538240,161.0412,0.00000,0.0000,Ph-Cl,3,12,24,...,-0.709510,-0.700082,11.090143,100,56,8,36,104,3.850000e-08,0.000004
68581,1188.302573,1.399781,1001.700216,150.9874,0.00000,0.0000,Pr-F,3,4,24,...,-0.398622,-0.155104,10.184892,110,47,8,55,119,3.360000e-08,0.000004
68582,1506.660363,1.645811,1493.296496,161.4001,0.01108,0.0000,SO3H,10,42,46,...,-2.389941,-1.681765,14.747027,115,38,27,50,126,-5.560000e-08,-0.000007


<h1>Normalizing</h1>

Normalizing functional groups

In [9]:
def check_in(pattern : str, loop : list or np.ndarray or None = df.functional_groups):
    return [int(pattern in str(x).split('-') )for x in loop]

def count_in(pattern : str, loop : list or np.ndarray or None = df.functional_groups):
    return [str(x).count(pattern) for x in loop]

compounds = set(['SO3H','COOH','NH2','OH','CN','F','OMe','NHMe','NO2','Pr','Cl','OEt','Ph','Br','OPr','HCO','Et','Me','H','I'])
molecules = set(['N','O','C'])

func_data = {
    f'funccheck_{compound}':check_in(compound)
    for compound in compounds
}
func_data.update({
    f'funccount_{molecule}':count_in(molecule)
    for molecule in molecules
})
func_data.update({
    'num_func': [int('-' in str(x)) for x in df.functional_groups]
})

df.functional_groups = df.functional_groups.astype("category").cat.codes
func_df = pd.DataFrame(func_data);func_df


,funccheck_COOH,funccheck_Pr,funccheck_OPr,funccheck_Br,funccheck_OMe,funccheck_Ph,funccheck_SO3H,funccheck_OEt,funccheck_F,funccheck_I,...,funccheck_Cl,funccheck_HCO,funccheck_CN,funccheck_NHMe,funccheck_NO2,funccheck_NH2,funccount_C,funccount_N,funccount_O,num_func
0,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,3,1
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68579,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68580,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1
68581,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
68582,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


Add more feature

In [10]:
df.insert(
    loc=0,
    column='difatom',
    value=df['sumatom']-df['C+O+H']
)
df.insert(
    loc=0,
    column='surface_to_volume',
    value=df['surface_area']/df['volume']
)
df.insert(
    loc=0,
    column='cubic_surface_area',
    value=((df['volume']**(1/3))**2)*6
)
df

,cubic_surface_area,surface_to_volume,difatom,volume,density,weight,surface_area,void_fraction,void_volume,functional_groups,...,Lattice7,Lattice8,Lattice9,C+O+H,C,O,H,sumatom,mean_charge,sum_charge
0,1183.275571,0.217949,21,2769.503842,1.326090,2211.697211,603.6100,0.13794,0.1040,115,...,-0.600769,-3.410680,18.641128,173,71,35,67,194,5.150000e-09,0.000001
1,635.410072,0.723515,2,1089.818728,1.178856,773.687960,788.5000,0.14874,0.1262,303,...,0.010529,0.148108,10.630959,80,32,14,34,82,-3.660000e-08,-0.000003
2,1016.520988,0.653696,5,2205.198301,0.982408,1304.638720,1441.5300,0.21814,0.2220,139,...,-0.078915,-6.446998,18.164659,107,56,23,28,112,4.460000e-08,0.000005
3,1500.458542,0.614604,6,3954.659761,0.647909,1543.027680,2430.5500,0.37094,0.5725,373,...,-2.470500,-0.012234,22.470616,158,72,24,62,164,-1.830000e-08,-0.000003
4,1400.442353,0.429068,17,3565.914939,0.910268,1954.749656,1530.0200,0.33337,0.3662,200,...,0.012840,-4.025383,22.428988,165,68,28,69,182,2.200000e-08,0.000004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68579,579.449866,0.173410,2,949.067112,1.318868,753.787520,164.5781,0.00000,0.0000,360,...,4.211774,-0.032201,15.139554,84,38,8,38,86,-3.490000e-08,-0.000003
68580,678.367236,0.133957,4,1202.182553,1.440028,1042.538240,161.0412,0.00000,0.0000,344,...,-0.709510,-0.700082,11.090143,100,56,8,36,104,3.850000e-08,0.000004
68581,673.135676,0.127061,9,1188.302573,1.399781,1001.700216,150.9874,0.00000,0.0000,366,...,-0.398622,-0.155104,10.184892,110,47,8,55,119,3.360000e-08,0.000004
68582,788.548045,0.107124,11,1506.660363,1.645811,1493.296496,161.4001,0.01108,0.0000,380,...,-2.389941,-1.681765,14.747027,115,38,27,50,126,-5.560000e-08,-0.000007


One-hotting metal/organic linkers and topology

In [11]:
metal_linker_int = df['metal_linker']-1
metal_one_hot = to_categorical(metal_linker_int,num_classes=12,dtype='int8')
metal_onehot_df = pd.DataFrame(metal_one_hot,columns=['ml_' + str(num) for num in range(1,13)])
print(metal_onehot_df.shape)
display(metal_onehot_df.head(2))
#---------------------------------------
org1_int = df['organic_linker1']-1
org1_one_hot = to_categorical(org1_int,num_classes=59,dtype='int8')
org1_onehot_df = pd.DataFrame(org1_one_hot,columns=['ol1_' + str(num) for num in range(1,60)])
print(org1_onehot_df.shape)
display(org1_onehot_df.head(2))
#--------------------------------------
org2_int = df['organic_linker2']-1
org2_one_hot = to_categorical(org2_int,num_classes=59,dtype='int8')
org2_onehot_df = pd.DataFrame(org2_one_hot,columns=['ol2_' + str(num) for num in range(1,60)])
print(org2_onehot_df.shape)
display(org2_onehot_df.head(2))
#--------------------------------------
top_int = df['topology']
top_one_hot = to_categorical(top_int,dtype='int8')
top_onehot_df = pd.DataFrame(top_one_hot,columns=['top_' + str(num) for num in range(0,11)])
print(top_onehot_df.shape)
display(top_onehot_df.head(2))

(68584, 12)


,ml_1,ml_2,ml_3,ml_4,ml_5,ml_6,ml_7,ml_8,ml_9,ml_10,ml_11,ml_12
0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0


(68584, 59)


,ol1_1,ol1_2,ol1_3,ol1_4,ol1_5,ol1_6,ol1_7,ol1_8,ol1_9,ol1_10,...,ol1_50,ol1_51,ol1_52,ol1_53,ol1_54,ol1_55,ol1_56,ol1_57,ol1_58,ol1_59
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


(68584, 59)


,ol2_1,ol2_2,ol2_3,ol2_4,ol2_5,ol2_6,ol2_7,ol2_8,ol2_9,ol2_10,...,ol2_50,ol2_51,ol2_52,ol2_53,ol2_54,ol2_55,ol2_56,ol2_57,ol2_58,ol2_59
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


(68584, 11)


,top_0,top_1,top_2,top_3,top_4,top_5,top_6,top_7,top_8,top_9,top_10
0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0


<h1>Feature selection</h1>

In [12]:
df_col = dict(
    df_geometry_col = [
    'surface_area',
    'void_fraction',
    'density',
    'void_volume',
    'weight',
    'volume',
    ],
    df_function_col = [
    'functional_groups',
    'metal_linker',
    'organic_linker1',
    'organic_linker2',
    'topology',
    'CO2/N2_selectivity',
    'heat_adsorption_CO2_P0.15bar_T298K',
    ],
    df_cif_col = [
    'cell_length_a',
    'cell_length_b',
    'cell_length_c',
    'cell_angle_alpha',
    'cell_angle_beta',
    'cell_angle_gamma',
    'sum_charge',
    # 'mean_charge',
    ],
    df_xyz_col = [
    'Lattice1', 
    # 'Lattice2',
    # 'Lattice3', 
    'Lattice4', 
    'Lattice5', 
    # 'Lattice6', 
    'Lattice7', 
    'Lattice8',
    'Lattice9', 
    # 'C+O+H', 
    'C', 
    'O', 
    'H', 
    'sumatom',
    ],
    df_add_col = [
    'surface_to_volume',
    'difatom',
    'cubic_surface_area'
    ]
)

df_selected = df[sum(df_col.values(),[])]
print(df_selected.shape)
df_selected = pd.concat([df_selected,metal_onehot_df,org1_onehot_df,org2_onehot_df,top_onehot_df,func_df],axis=1)
print(df_selected.shape);df_selected.head()

(68584, 33)
(68584, 198)


,surface_area,void_fraction,density,void_volume,weight,volume,functional_groups,metal_linker,organic_linker1,organic_linker2,...,funccheck_Cl,funccheck_HCO,funccheck_CN,funccheck_NHMe,funccheck_NO2,funccheck_NH2,funccount_C,funccount_N,funccount_O,num_func
0,603.61,0.13794,1.326090,0.1040,2211.697211,2769.503842,115,10,44,57,...,0,0,0,0,0,0,0,0,1,1
1,788.50,0.14874,1.178856,0.1262,773.687960,1089.818728,303,2,22,24,...,0,0,0,0,0,0,1,0,3,1
2,1441.53,0.21814,0.982408,0.2220,1304.638720,2205.198301,139,9,17,24,...,0,0,0,0,0,0,0,0,1,1
3,2430.55,0.37094,0.647909,0.5725,1543.027680,3954.659761,373,9,7,23,...,0,0,0,0,1,0,0,1,1,1
4,1530.02,0.33337,0.910268,0.3662,1954.749656,3565.914939,200,10,53,55,...,0,0,0,0,0,1,0,1,0,0


Feature selection with BoostARoota

In [13]:
# unimportant_col = ["top_7","ol2_59","ol2_39","ol2_37","ol2_34","ol2_33","ol2_32","ol1_58","ol1_57","ol1_37","ol1_33","ol1_31","ml_11","ml_8","ml_7","ml_6","ml_5"]
# df_selected = df_selected.drop(unimportant_col,axis=1)

<h2>convert to array</h2>

In [14]:
df_label_array = np.array(df_label)
df_array = np.array(df_selected)
f'{df_label_array.shape} : {df_array.shape}'

'(68584,) : (68584, 198)'

<h2>train-test split</h2>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_array,df_label_array,test_size=0.1,random_state=123)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.15,random_state=123)
print(f'train : {X_train.shape}\nval : {X_val.shape}\ntest : {X_test.shape}')

In [15]:
X_train, X_test, y_train, y_test = train_test_split(df_array,df_label_array,test_size=0.1,random_state=123)
print(f'train : {X_train.shape}\ntest : {X_test.shape}')

train : (61725, 198)
test : (6859, 198)


<h1> Model training </h1>

<h1>Comparing</h1>

In [ ]:
pred_df_plot = pred_df[pred_df[0]>400]
tomuch = len(pred_df_plot[(pred_df_plot[1]-pred_df_plot[0]).astype(int)<0])
tolittle = len(pred_df_plot[(pred_df_plot[1]-pred_df_plot[0]).astype(int)>0])
equal = len(pred_df_plot[(pred_df_plot[1]-pred_df_plot[0]).astype(int)==0])
plt.bar(['tomuch','tolittle','equal'],[tomuch,tolittle,equal])

<h1>Evaluation</h1>

In [ ]:
testset = pd.read_csv('Datasets/cif_xyz_nonfg_test.csv')
print(testset.shape);testset.head()

In [ ]:
test_func_data = {
    f'funccheck_{compound}':check_in(compound,loop=testset.functional_groups)
    for compound in compounds
}
test_func_data.update({
    f'funccount_{molecule}':count_in(molecule,loop=testset.functional_groups)
    for molecule in molecules
})
test_func_data.update({
    'num_func': [int('-' in str(x)) for x in testset.functional_groups]
})
testset.functional_groups = testset.functional_groups.astype("category").cat.codes
test_func_df = pd.DataFrame(test_func_data);test_func_df.head(5)

In [ ]:
test_metal_linker_int = testset['metal_linker']-1
test_metal_one_hot = to_categorical(test_metal_linker_int,num_classes=12,dtype='int8')
test_metal_onehot_df = pd.DataFrame(test_metal_one_hot,columns=['ml_' + str(num) for num in range(1,13)])
display(test_metal_onehot_df.head(3))
#-------------------------------------------------
test_org1_int = testset['organic_linker1']-1
test_org1_one_hot = to_categorical(test_org1_int,num_classes=59,dtype='int8')
test_org1_onehot_df = pd.DataFrame(test_org1_one_hot,columns=['ol1_' + str(num) for num in range(1,60)])
display(test_org1_onehot_df.head(3))
#-------------------------------------------------
test_org2_int = testset['organic_linker2']-1
test_org2_one_hot = to_categorical(test_org2_int,num_classes=59,dtype='int8')
test_org2_onehot_df = pd.DataFrame(test_org2_one_hot,columns=['ol2_' + str(num) for num in range(1,60)])
display(test_org2_onehot_df.head(3))
#-------------------------------------------------
test_top_int = testset['topology']
test_top_one_hot = to_categorical(test_top_int,dtype='int8')
test_top_onehot_df = pd.DataFrame(test_top_one_hot,columns=['top_' + str(num) for num in range(0,11)])
display(test_top_onehot_df.head(3))

In [ ]:
testset.insert(
    loc=33,
    column='difatom',
    value=testset['sumatom']-testset['C+O+H']
)
testset.insert(
    loc=0,
    column='surface_to_volume',
    value=testset['surface_area']/testset['volume']
)
testset.insert(
    loc=0,
    column='cubic_surface_area',
    value=((testset['volume']**(1/3))**2)*6
)
testset.void_volume = testset.void_volume**2
testset.surface_area = (testset.surface_area**2)

In [ ]:
testset = testset[sum(df_col.values(),[])]
testset = pd.concat([testset,
test_metal_onehot_df,
test_org1_onehot_df,
test_org2_onehot_df,
test_top_onehot_df,
test_func_df
],axis=1);testset.head(3)

drop unimportant

In [ ]:
# testset = testset.drop(unimportant_col,axis=1)

check validation

In [ ]:
print(f'{len(testset.columns)} : {len(df_selected.columns)}')
if(len(testset.columns)==len(df_selected.columns)):
    print(all(testset.columns == df_selected.columns))
    print(all(testset.dtypes == df_selected.dtypes))
else:print(False)

In [ ]:
testset_array = np.array(testset);testset.shape

predict

In [ ]:
# loadModel = xgb.Booster()
# loadModel.load_model('2914.json')

In [ ]:
test_pred = xg_reg1.predict(testset_array);test_pred

In [ ]:
# test_pred = xg_reg1.predict(testset_array);test_pred

convert to dataframe

In [ ]:
submission = pd.DataFrame({
    "id": [str(i) for i in range(68614,85614)],
    "CO2_working_capacity [mL/g]":test_pred
    })
submission.head()

save csv

In [ ]:
submission.to_csv('submission.csv',index=False,float_format='%.7f')

--------------------------------------------------------------------------------------------